In [2]:
import os

# folder_path = 'C:/Users/DuYih/Desktop/PVC/Single_Sample'

# output_file = open('output.csv', 'w')
# output_file.write('File,Line\n')  # 输出文件头

# for dirpath, dirnames, filenames in os.walk(folder_path):
#     for filename in filenames:
#         if filename == 'classification.txt':
#             filepath = os.path.join(dirpath, filename)
#             with open(filepath, 'r') as f:
#                 for line in f:
#                     if 'Verrucomicrobia' or 'Planctomycetales' or 'Chlamydiae' in line:
#                         output_file.write('{},{}'.format(filepath, line))
                        
# output_file.close()


# Verrucomicrobia in maps

In [3]:
def filter_samples(sample_data_path, keyword="Verrucomicrobia"):
    with open(sample_data_path, 'r', encoding='utf-8') as f:  # specify the encoding
        content = f.read()

    samples = content.split(">")[1:]  
    filtered_samples = [sample for sample in samples if keyword in sample]

    return filtered_samples


In [4]:
verrucomicrobia_samples = filter_samples('samples.info')

MemoryError: 

In [12]:
def filter_samples(sample_data_path, keyword="Verrucomicrobia"):
    with open(sample_data_path, 'r', encoding='utf-8') as f:
        sample = ""
        for line in f:
            if line.startswith(">"):
                if keyword in sample:
                    yield sample
                sample = line
            else:
                sample += line
        if keyword in sample:  # don't forget the last sample
            yield sample

# with open('verrucomicrobia_samples.txt', 'w') as f:
#     for sample in filter_samples('samples.info'):
#         f.write(">" + sample) 

with open('verrucomicrobia_samples.txt', 'w', encoding='utf-8') as f:
    for sample in filter_samples('samples.info'):
        f.write(sample) 


In [4]:
import pandas as pd

# Load the data
data = pd.read_csv("verrucomicrobia_samples.txt", sep="\t")

# Display the first few rows of the dataframe
data.head()


,>SRS994464
0,sample_center_name=
1,sample_alias=NASH_50
2,sample_TAXON_ID=408170
3,sample_SCIENTIFIC_NAME=human gut metagenome
4,sample_XREF_LINK=bioproject: 286838)


In [22]:
# Initialize an empty dictionary to store the data
data_dict = {}
# Initialize an empty dictionary to store the current sample's data
current_sample = {}
# Initialize an empty string to store the current sample's ID
current_sample_id = ""

# Iterate over each row in the data
for row in data.iterrows():
    # Get the row's data
    row_data = row[1][0]
    # If the row's data starts with ">", it's a new sample ID
    if row_data.startswith(">"):
        # If we have data for the current sample, add it to the data dictionary
        if current_sample:
            data_dict[current_sample_id] = current_sample
        # Reset the current sample's data
        current_sample = {}
        # Set the current sample ID to the new sample ID (minus the ">")
        current_sample_id = row_data[1:]
    # If the row's data contains "=", it's a key-value pair
    elif "=" in row_data:
        # Split the key and value
        key, value = row_data.split("=", 1)
        # Add the key-value pair to the current sample's data
        current_sample[key] = value

# Add the last sample's data to the data dictionary
if current_sample:
    data_dict[current_sample_id] = current_sample

# Convert the data dictionary to a DataFrame
samples_df = pd.DataFrame.from_dict(data_dict, orient="index")

samples_df.to_csv("verrucomicrobia_samples.csv")

In [1]:
samples_with_coordinates = samples_df.dropna(subset=['sample_geo_loc_name'])

# Remove rows where geo location is 'not provided'
samples_with_coordinates = samples_with_coordinates[samples_with_coordinates['sample_geo_loc_name'] != 'not provided']

samples_with_coordinates.to_csv("verrucomicrobia_samples_with_coordinates.csv")


NameError: name 'samples_df' is not defined

In [6]:
import requests
from bs4 import BeautifulSoup

# 发起HTTP请求获取网页内容
url = "https://microbeatlas.org/index.html?action=sample_detail&sid=DRS016395&rid=DRR019542"  # 替换成你要爬取的网站URL
response = requests.get(url)
html_content = response.content

# 解析HTML内容
soup = BeautifulSoup(html_content, "html.parser")

# 查找特定内容
title = soup.title.text  # 获取网页标题
links = soup.find_all("a")  

# 输出结果
print("网页标题:", title)
print("链接列表:")
for link in links:
    print(link.get("href"))
filtered_links = [link.get("href") for link in links if link.get("href").startswith(("http://", "https://"))]


网页标题: Microbe Atlas Project (MAP)
链接列表:
javascript:open_landing();
javascript:open_taxa();
javascript:open_samples();
javascript:open_projects();
javascript:open_publications();
javascript:open_habitats();
javascript:open_mapseq();
#
javascript:open_about();
javascript:open_help();
javascript:open_contributors();
javascript:open_contact();
javascript:open_download();
https://github.com/meringlab/MicrobeAtlasWebsite/issues
http://www.uzh.ch
https://nccr-microbiomes.ch/
http://www.isb-sib.ch
javascript:open_contributors();
javascript:open_contact();
https://github.com/jfmrod/MAPseq


In [9]:
from Bio import Entrez

def download_sample_data(accession_number):
    Entrez.email = 'u7457260@anu.edu.au'  
    handle = Entrez.efetch(db='nuccore', id=accession_number, rettype='fasta', retmode='text')
    data = handle.read()
    handle.close()
    return data

accession_number = 'ERS1882911'
sample_data = download_sample_data(accession_number)
from Bio import SeqIO

def calculate_species_proportion(fasta_data, target_species):
    sequences = SeqIO.parse(fasta_data, 'fasta')
    total_count = 0
    target_count = 0

    for sequence in sequences:
        description = sequence.description
        if target_species in description:
            target_count += 1
        total_count += 1

    proportion = target_count / total_count * 100
    return proportion

target_species = 'Verrucomicrobia'
proportion = calculate_species_proportion(sample_data, target_species)
print(f"The proportion of {target_species} in the sample is: {proportion}%")


HTTPError: HTTP Error 400: Bad Request